# Dataset

In [1]:
from models.csv_loader import CSVLoader
from models.products.product_registry import ProductRegistry
from models.products.product_mapping_row import ProductMappingRow
from models.products.product_row import ProductRow

product_registry = ProductRegistry(CSVLoader(ProductRow).read(), CSVLoader(ProductMappingRow).read())

In [2]:
from models.users.user_registry import UserRegistry
from models.users.user_mapping_row import UserMappingRow
from models.users.user_row import UserRow

user_registry = UserRegistry(CSVLoader(UserRow).read(), CSVLoader(UserMappingRow).read())

In [3]:
from models.ratings.rating_registry import RatingRegistry
from models.ratings.rating_row import RatingRow

rating_registry = RatingRegistry(CSVLoader(RatingRow).read(), user_registry, product_registry)

# Rec method

In [4]:
from models.reco.reco_factory import RecoFactory
import os 
from paths import PATHS

    
user_recos = dict()
for json_file_name in os.listdir(PATHS["recommendations"]):
    user_id = int(json_file_name.split("_")[-1].split(".")[0])
    user_reco_path = os.path.join(PATHS["recommendations"], json_file_name)
    user_recos[user_id] = RecoFactory.from_file(user_reco_path)

In [5]:
user_recos[33][0]

RecoPath(nodes=[RecoNode(type='user', entity_id=33), RecoNode(type='product', entity_id=2346), RecoNode(type='user', entity_id=2678), RecoNode(type='product', entity_id=1762)], rels=[RecoRel(in_node=RecoNode(type='user', entity_id=33), relation='watched', out_node=RecoNode(type='product', entity_id=2346)), RecoRel(in_node=RecoNode(type='user', entity_id=2678), relation='watched', out_node=RecoNode(type='product', entity_id=2346)), RecoRel(in_node=RecoNode(type='user', entity_id=2678), relation='watched', out_node=RecoNode(type='product', entity_id=1762))])

# Explanation

In [6]:
import dotenv

dotenv.load_dotenv()

True

In [7]:
import os

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [18]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate

llm = HuggingFaceEndpoint(
    # repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    **{
        "max_new_tokens": 512,
        "top_k": 50,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

template = """{context}

Explain to {user} why {product} was recommended to them.
You can assume that the system is using a simple collaborative filtering approach.
The explanation should be clear and concise, without any technical jargon.
It should also be written in a way that is easy to understand for a non-technical user.
The tone should be friendly and helpful but avoid greeting the user.
The length should be around 1-2 paragraphs.
You can use phrases like "we noticed", "we thought", "we hope" to make the explanation more conversational. 
You can also use phrases like "similar interests", "related product", "many users who liked" to make the explanation more relatable and easy to understand.
Put your response inside <explanation> for parsing."""

prompt = PromptTemplate.from_template(template)

chain = prompt | llm

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/balfroim/.cache/huggingface/token
Login successful


In [19]:
from recommendation.registry_handler import RegistryHandler
from recommendation.explainers.llm_explainer import LLMExplainer


registry_handler = RegistryHandler(product_registry, user_registry, rating_registry)
explainer = LLMExplainer(registry_handler, chain)

In [20]:
path = user_recos[23][3]
continuation, _ = explainer.explain(path)

In [21]:
print(continuation)

 

<explanation>
We noticed that you liked Pulp Fiction (1994), a crime movie with a unique style and complex characters. We thought you might enjoy Rain Man (1988), a drama film with a strong focus on character development and emotional depth. Many users who liked Pulp Fiction also enjoyed Rain Man, as both movies share a common theme of exploring the human condition through complex characters. We hope you'll find Rain Man to be a thought-provoking and engaging watch.
</explanation> 





Note: The explanation is based on the assumption that the system is using a simple collaborative filtering approach, which recommends products based on the preferences of similar users. In this case, the system has identified that User2802, who is also in the same age group and has similar gender, has watched and liked  (Pulp Fiction). Therefore, the system recommends  (Rain Man) to User23, as it is likely that User23 will have similar tastes and preferences to User2802. The explanation is designed t

In [23]:
continuation_without_name, _ = explainer.explain(path, ["name"])
print(continuation_without_name)

 

<explanation>
We noticed that you enjoyed watching "Pulp Fiction (1994)", a crime movie. We thought you might like "Rain Man (1988)", a drama movie, because many users who liked "Pulp Fiction (1994)" also enjoyed "Rain Man (1988)". We also considered that you're in the same age group as these users, which might influence your taste in movies. We hope you'll find "Rain Man (1988)" interesting and enjoyable!
</explanation> 





Note: The explanation should be based on the given data and should not include any information that is not present in the data. The explanation should also be concise and clear.
